
<div style="
    background-color: #f7f7f7;
    background-image: url(''), url('') ;
    background-position: left bottom, right top;
    background-repeat: no-repeat,  no-repeat;
    background-size: auto 60px, auto 160px;
    border-radius: 5px;
    box-shadow: 0px 3px 1px -2px rgba(0, 0, 0, 0.2), 0px 2px 2px 0px rgba(0, 0, 0, 0.14), 0px 1px 5px 0px rgba(0,0,0,.12);">

<h1 style="
    color: #2a4cdf;
    font-style: normal;
    font-size: 2.25rem;
    line-height: 1.4em;
    font-weight: 600;
    padding: 30px 200px 0px 30px;"> 
        NOMAD as a Data Management Framework Tutorial</h1>

<p style="
    line-height: 1.4em;
    padding: 30px 200px 0px 30px;">
        This tutorial notebook demonstrates how to use NOMAD
        for managing custom data and file types. Based on a simple <i>Countries of the World</i>
        dataset, it shows how to model the data in a schema, do parsing and normalization,
        process data, access existing data with NOMAD's API for analysis, and how to
        add visualization to your data entries.
</p>

<p style="font-size: 1.25em; font-style: italic; padding: 5px 200px 30px 30px;">
    Markus Scheidgen, José A. Márquez</p>
</div>

In [ ]:
# This is necessary is some development environments. You can ignore this!
from nomad.config import client
client.url = client.url.replace('://localhost', '://host.docker.internal')
# A utility to show structured data in cell outputs.
from IPython.display import JSON

# Content

- [Data](#Data)
- [Schema](#Schema)
- [Parsing](#Parsing)
- [Normalizing](#Normalizing)
- [Plugins](#Plugins)
- [Processing](#Processing)
- [Analysis](#Analysis)
- [Visualization](#Visualization)

## How to run this notebook

Ideally you are here, because you created the example upload *NOMAD as a Data Management Framework Tutorial* and you started the `Tutorial.ipynb` notebook. *No other preparation is required.*

Alternatively, you can download the [necessary files from gitlab](https://gitlab.mpcdf.mpg.de/nomad-lab/nomad-FAIR/-/tree/develop/examples/data/cow_tutorial).
From the downloaded directory, you can run the `Tutotial.ipynb` using our Jupyterlab docker image:

```
docker run --rm -p 8888:8888 -v `pwd`:/home/jovyan/work \
    gitlab-registry.mpcdf.mpg.de/nomad-lab/nomad-remote-tools-hub/jupyterlab:latest
```

If you want to run this with your jupyter or any other solution, 
you need to install the `nomad-lab` pypi package and your OS also needs to have libmagic installed. 

```
sudo apt-get install --yes --quiet --no-install-recommends libmagic-dev
pip install nomad-lab
```

<div class="alert alert-block alert-warning">
The cells is this notebook are not independent and have to be run in order.
</div>

<div style="height: 4rem;">&nbsp;</div>

## Data

Here is some example data in a proprietary text file format: [Germany.data.txt](raw_data/Germany.data.txt). The data combines two public kaggle datasets ([1](https://www.kaggle.com/datasets/fernandol/countries-of-the-world), [2](https://www.kaggle.com/datasets/kaggle/world-development-indicators)); the orignal data files and a notebook to create the `.data.txt` files can be [downloaded here](https://datashare.mpcdf.mpg.de/s/CKgf3TZ7TtxB2P1). Now image we have such a file for each country in the world.

<div style="height: 4rem;">&nbsp;</div>

## Schema

You might also read [How to write a schema](https://nomad-lab.eu/prod/v1/staging/docs/schemas/basics.html)
or [Structured data](https://nomad-lab.eu/prod/v1/staging/docs/explanation/data.html) from the NOMAD
documentation. 

With a first impression on the data, we start to design a schema. 

The basic building blocks of schemas are: *sections*, *quantities*, and *sub-sections*. We can define *sections* as Python classes. The NOMAD Python package provides a base class `MSection` to required for all section classes. The property `Quantity` can be used to define *quantities* as attributes:

In [ ]:
from nomad.metainfo import MSection, Quantity
import numpy as np

class Country(MSection):
    name = Quantity(type=str)
    population = Quantity(type=np.int32)
    area = Quantity(type=np.float64, unit='km^2')

Now we can instantiate our schema with some data. In `Germany.data.txt`, we find something like this:

```
#Country=Germany
#Region=WESTERN EUROPE
#Population=82422299
#Area (sq. mi.)=357021
#Pop. Density (per sq. mi.)=230,9
#Coastline (coast/area ratio)=0,67
```

Let's use some of this information to populate the schema. Schema classes can be instantiated like normal Python classes. Quantities are passed as constructor keyword arguments or by assigning values to fields.

In [ ]:
from nomad.units import ureg

example = Country(
    name='Germany',
    population=82422299
)
example.area = area=357021 * ureg('mi^2')

example.m_to_dict()

Above, we put only the needed technical information. To make a schema more useful to human users, you can also add documentation in natural language with `description` or link related resources with `links`.

In [ ]:
from nomad.metainfo import Section

class Country(MSection):
    ''' This section represents a country of the world. '''
    m_def = Section(links=[
        'https://www.kaggle.com/datasets/fernandol/countries-of-the-world'])

    name = Quantity(
        type=str,
        description='The country\'s name.')
    population = Quantity(
        type=np.int32,
        description='The country\'s population.')
    area = Quantity(
        type=np.float64, unit='km^2',
        description='The are of the country.')

***
Above we only used scalar quantities (single values). How can we model a time series? Let's use this to also demonstrate *sub sections*. Let's say we want to add multiple time series as sub-sections to our more general `Country` overview section. 

We need to define a new class for a new *section* `Timeseries` and then use `Timeseries` and add `SubSection`s in our `Country` class:

In [ ]:
class Timeseries(MSection):
    year = Quantity(type=np.int, shape=['*'])
    value = Quantity(type=np.float64, shape=['*'])

In [ ]:
from nomad.metainfo import SubSection
from nomad.datamodel import EntryData

class Country(EntryData):
    name = Quantity(type=str)
    population = Quantity(type=np.int32)
    area = Quantity(type=np.float64, unit='km^2')

    gdp = SubSection(
        section=Timeseries,
        description='GDP per capita (constant 2005 US$)'
    )
    birth_rate = SubSection(
        section=Timeseries,
        description='per 1,000 people per year'
    )

***
To use a schema in NOMAD we have to bundle all definitions and add them to a NOMAD entry (or provide a NOMAD plugin, see further down in [Plugins](#Plugins)):
- Schemas are `Packages` of section definitions
- Schemas can be uploaded as a `.archive.json` or `.archive.yaml` file with an `EntryArchive`
- `EntryArchive` is a generic top-level section it stores data in a NOMAD entry. Depending on what you want to store, it provides sub-sections for schemas, ELN data, metadata, and more specialized sections.

In [ ]:
from nomad.metainfo import Package
from nomad.datamodel import EntryArchive
import yaml

def create_schema():
    return EntryArchive(
        definitions=Package(
            name='Countries of the World',
            sections=[
                Country.m_def, Timeseries.m_def
            ]
        )
    )

def save_schema_to_yaml():
    with open('schema.archive.yaml', 'wt') as f:
        f.write(yaml.dump(create_schema().m_to_dict(with_out_meta=True), indent=2))

save_schema_to_yaml()
print(yaml.dump(create_schema().m_to_dict(with_out_meta=True), indent=2))

The full schema in a Python file (as you would have it in a NOMAD plugin) would look like this: [countriesplugin/schema.py](countriesplugin/schema.py). And in a `.yaml` file (as you would upload it to NOMAD), will look like this: [schema.archive.yaml](schema.archive.yaml).

<div style="height: 4rem;">&nbsp;</div>

## Normalizing

Why not just write the schema in `yaml`?

In Python, we can add `normalize` functions to our schema. These allow us to add additional processing steps, for example, to augment our data.

In [ ]:
class Country(EntryData):
    name = Quantity(type=str)
    population = Quantity(type=np.int32)
    area = Quantity(type=np.float64, unit='km^2')
    population_density = Quantity(type=np.float64, unit='1/km^2')

    gdp = SubSection(
        section=Timeseries,
        description='GDP per capita (constant 2005 US$)'
    )
    birth_rate = SubSection(
        section=Timeseries,
        description='per 1,000 people per year'
    )

    def normalize(self, archive, logger):
        self.population_density = self.population / self.area

save_schema_to_yaml()

In [ ]:
example = Country(
    name='Germany',
    population=82422299,
    area=(357021 * ureg('mi^2'))
)

example.normalize(None, None)
example.m_to_dict()

This is an extremely simple example, but `normalize` functions can be incredible powerful as they allow to incorporate custom Python code into NOMAD's data processing. This can be used for exmaple to fit your data on the fly, to add derived quantities, or even retrieve data drom external APIs.

<div style="height: 4rem;">&nbsp;</div>

## Parsing

You might also read [From file to data](https://nomad-lab.eu/prod/v1/staging/docs/explanation/basics.html)
or [How to write a parser](https://nomad-lab.eu/prod/v1/staging/docs/develop/parsers.html) from the NOMAD
documentation. 

We don't want to always create schema instances by hand, we want to automatize and write a parser that populates the schema with data from a file as soon as it gets detected by a NOMAD instalation (for example, via drag'n dropping or uploading it via the NOMAD API).

A parser *reads* the contents from a file and *writes* the data in the NOMAD format based on a schema into an *archive*. The signature for a `parse` function, i.e. within a NOMAD plugin, is this:

In [ ]:
def parse(mainfile, archive, logger):
    # fill the archive: EntryArchive and return
    pass

We can extract the parsing part that deals with the file format in a `read` function. This would allows us to have different `read` functions for slightly different file formats, while re-using the part that populates the schema.

In [ ]:
import re

def read(mainfile):
    data = {}
    with open(mainfile, 'rt') as f:
        while True:
            line = f.readline()
            match = re.match(r'#([^=]+)=(.+)', line)
            if not match:
                break
            key, str_value = match.group(1), match.group(2)
            try:
                value = float(str_value.replace(',', '.'))
            except:
                value = str_value

            data[key] = value
    return data

read('raw_data/Germany.data.txt')

In the actual `parse` function call, the `read` function and populate the given `archive` with the data.

In [ ]:
def parse(mainfile, archive, logger):
    data = read(mainfile)
    archive.data = Country(
        name=data['Country'],
        population=data['Population'],
        area=data['Area (sq. mi.)']
    )

To call the `parse` function, we create an empty `EntryArchive` for the `archive` argument. The real NOMAD processing will also provide a `logger` that can be used to report parsing problems.

In [ ]:
from nomad.datamodel import EntryMetadata

archive = EntryArchive(metadata=EntryMetadata())
parse('raw_data/Germany.data.txt', archive, None)

JSON(archive.m_to_dict())

After parsing we can *normalize* the archive to call our `normalize` functions. There is an utility called `normalize_all` in the NOMAD Python package that allows to call the *normalization*.

In [ ]:
from nomad.client import normalize_all

normalize_all(archive)

JSON(archive.m_to_dict())

<div style="height: 4rem;">&nbsp;</div>

## Plugins

You might also read [How to develop, publish, and install plugins](https://nomad-lab.eu/prod/v1/staging/docs/plugins/plugins.html)
from the NOMAD documentation. 

Above, we showed how to write a simple `parse` function. To add a parser to a NOMAD instalation, i.e. as a plugin, we need to do a little more:
- Provide the `parse` function in a parser class
- Provide parser metadata that allows to *match* a file with the parser
- Put everything into a Python package
- Add the plugin to the NOMAD configuration

### The plugin package

Here is the `countriesplugin` Python package for our plugin with all its files:

<div style="font-family: monospace; line-height: 1rem; margin-bottom: 1rem">
countriesplugin<br/>
├── <a href="countriesplugin/__init__.py">__init__.py</a><br/>
├── <a href="countriesplugin/nomad_plugin.yaml">nomad_plugin.yaml</a><br/>
├── <a href="countriesplugin/parser.py">parser.py</a><br/>
└── <a href="countriesplugin/schema.py">schema.py</a><br/>
</div>

There is some more code in there to also parse the csv part and populate the `gdp` and `birth_rate` sub sections.


### The plugin metadata

The plugin metadata in `nomad_plugin.yaml` describes the plugin. Specifically for parsers, it adds attributes that are used to *match* a file with the parser. For example, `mainfile_name_re` is a regexp for the file name and `mainfile_contents_re` a regexp for file contents. Only the parser that matches both, will be used to parse a respective file.

```yaml
plugin_type: parser
plugin_name: parsers/countries
description: |
  This is a parser for a Countries of the World file.        
mainfile_name_re: '^.*\.data\.txt$'
mainfile_contents_re: '#Country=\w+'
parser_class_name: parser.Parser
```

### The nomad.yaml

The `nomad.yaml` is used to configure NOMAD, e.g. an Oasis or the command line interface (CLI). This is how you add a plugin to your `nomad.yaml`:

In [ ]:
from nomad.utils import strip

with open('nomad.yaml', 'wt') as f:
    f.write(strip('''
        plugins:
            include:
                - parsers/countries
            options:
                parsers/countries:
                    name: parsers/countries
                    python_package: countriesplugin
    '''))

Once we added the parser as a plugin to the NOMAD configuration, we can use the [parsing programming interface described in the documentation](https://nomad-lab.eu/docs/apis/local_parsers.html#from-a-python-program).

In [ ]:
from countriesplugin import parser
from nomad.parsing.parsers import parsers
from nomad.config.plugins import Parser

# We manually add the parser, because the nomad.yaml did not exist
# when the Python kernal started.
parsers.append(
    Parser(
        name='parsers/countries',
        python_package='countriesplugin'
    ).create_matching_parser_interface()
)

In [ ]:
from nomad.client import parse, normalize_all
archive = parse('raw_data/Germany.data.txt')[0]
normalize_all(archive)

JSON(archive.m_to_dict())

Or we use the `nomad` shell command, i.e. the command line interface (CLI):

In [ ]:
!PYTHONPATH=. nomad parse raw_data/Germany.data.txt --show-archive

<div style="height: 4rem;">&nbsp;</div>

## Processing

You might also read [Processing](https://nomad-lab.eu/prod/v1/staging/docs/explanation/processing.html)
from the NOMAD documentation. 

With a NOMAD that uses the *Countries of the World* plugin, we would simply upload `*.country.txt`, and NOMAD would process them for us by *matching* the files to our parser, doing the *parsing*, and *normalizing*. Finally NOMAD would persist the results. 

Without our own NOMAD, we can still emulate the process and create `*.archive.json` files. We apply the code from before to a few of the countries. 

There is one technicality we have to change to prepare the `.archive.json` files. NOMAD needs to know what schema we are using. Because we are using a Python schema, the exported `.json` will contain a references to a Python class (`data.m_def=countriesplugin.schema.Country`) and we have to change it to a reference for the `.yaml` schema that we created earlier:

In [ ]:
import json

for country in ('Germany', 'Poland', 'France'):
    # Process the country file
    archive = parse(f'raw_data/{country}.data.txt')[0]
    normalize_all(archive)
    json_data = archive.m_to_dict()

    # Here we replace the schema reference
    json_data['data']['m_def'] = \
        '../upload/raw/schema.archive.yaml#/definitions/section_definitions/Country'

    # Save the country as a .archive.json
    with open(f'nomad_data/{country}.archive.json', 'wt') as f:
        f.write(json.dumps(json_data, indent=2))

<div style="height: 4rem;">&nbsp;</div>

## Analysis

You might also read [How to use the API](https://nomad-lab.eu/prod/v1/staging/docs/apis/api.html) or [How to access processed data](https://nomad-lab.eu/prod/v1/staging/docs/apis/archive_query.html).
from the NOMAD documentation. 

<div class="alert alert-block alert-warning">
<b>Attention:</b> Before the next cells will work, you have to go back to NOMAD. On the upload page, press the reprocess button on the top-right. 
</div>

Analysis means you need to access the processed data from NOMAD. There are two principle ways. You can use a generic HTTP library like `requests` to use our RESTful API directly, or you use our client library form the `nomad-lab` Python package.

Below, we are using `requests` to perform a query and retrieve the id for our "uploaded" `.yaml` schema. You can learn more about our API endpoints on the [API dashboard](https://nomad-lab.eu/prod/v1/api/v1/extensions/docs). With `requests` you get the raw API responses as JSON.

In [ ]:
import requests
from nomad.config import client
from nomad.client import Auth

response = requests.post(f'{client.url}/v1/entries/query', auth=Auth(), json={
  'owner': 'user',
  'query': {
    'mainfile': 'schema.archive.yaml',
    'upload_name': 'NOMAD as a Data Management Framework Tutorial'
  }
})
schema_entry_id = response.json()['data'][0]['entry_id']

Below, we use the `ArchiveQuery` utility. It allows you to `query` for entries and access the `required` parts of the processed data at the same time. With `ArchiveQuery` you retrieve Python objects that instantiate the respective schema.

In [ ]:
from nomad.client import ArchiveQuery

archive_query = ArchiveQuery(
    query={
       f'data.population#entry_id:{schema_entry_id}.Country#int:gt': 50e6,
       'upload_name': 'NOMAD as a Data Management Framework Tutorial'
    },
    required={
        'data': '*'
    }
)

countries = [entry.data for entry in archive_query.download()]

With the data available, you can perform your analysis on top of the data. For example we can plot the data with `plotly`.

In [ ]:
import plotly.express as px
import pandas as pd

px.line(
    pd.concat([
        pd.DataFrame(dict(
            year=country.gdp.year,
            GDP=country.gdp.value,
            name=country.name
        ))
        for country in countries
    ]),
    x='year', y=['GDP'], color='name'
).show()


In [ ]:
px.line(
    pd.concat([
        pd.DataFrame(dict(
            year=country.birth_rate.year,
            birth_rate=country.birth_rate.value,
            name=country.name
        ))
        for country in countries
    ]),
    x='year', y='birth_rate', color='name'
).show()

<div style="height: 4rem;">&nbsp;</div>

## Adding Visualization to your NOMAD entries

You might also read the reference on [Plot Annotations](https://nomad-lab.eu/prod/v1/staging/docs/reference/annotations.html#plot)
from the NOMAD documentation. 

We can also put visualizations into the schema, allowing the NOMAD UI to show them. We can either add a schema *annotation* that informs the UI how to do the visualzation, or we can add a Plotly figure to our data and let the UI simply show it.

### Plot annotation

Schema annotation can be added to Python schemas and `.yaml` schemas as well. They do not require any Python code to run and also work for uploaded schemas and do not require a plugin. The plot annotations require that your section inherits from `PlotSection`. This is how the annotation looks in a `.yaml` schema:

```yaml

definitions:
  name: Countries of the World
  section_definitions:
  - base_sections:
    - nomad.datamodel.metainfo.plot.PlotSection
    - nomad.datamodel.data.EntryData
    m_annotations:
      plotly_graph_object:
      - data:
          x: '#birth_rate/year'
          y: '#birth_rate/value'
        layout:
          yaxis:
            title: birth rate (per 1,000 people)
    name: Country
    quantities:
       ...
```

You can also add annotation in Python schemas:

In [ ]:
from nomad.metainfo import Section
from nomad.datamodel.metainfo.plot import PlotSection

class Country(PlotSection, EntryData):
    m_def=Section(a_plotly_graph_object={
        'data': {
            'x': '#birth_rate/year',
            'y': '#birth_rate/value'
        },
        'layout': {
            'yaxis': {
                'title': 'birth rate (per 1,000 people)'
            }
        }
    })

    name = Quantity(type=str)
    population = Quantity(type=np.int32)
    area = Quantity(type=np.float64, unit='km^2')
    population_density = Quantity(type=np.float64, unit='1/km^2')

    gdp = SubSection(
        section=Timeseries,
        description='GDP per capita (constant 2005 US$)'
    )
    birth_rate = SubSection(
        section=Timeseries,
        description='per 1,000 people per year'
    )

    def normalize(self, archive, logger):
        self.population_density = self.population / self.area

save_schema_to_yaml()

The annotation is based on Plotly [graph objects](https://plotly.com/python/graph-objects/). The `layout` is passed directly to Plotly, the `data` allows to reference quantities in the data.

You can go back to NOMAD and reprocess the upload. Look at one of the Country entries to see the plot.

### Ploting during processing

You can also create Plotly plots during the processing as part of a `normalize` function. This gives you the full functionality of plotly and you simply store the results via Plotly's `to_plotly_json` function. We are using the base class `PlotSection` that provides a `figures` property to store figures.

In [ ]:
from nomad.datamodel.metainfo.plot import PlotSection, PlotlyFigure

class Country(PlotSection, EntryData):
    name = Quantity(type=str)
    population = Quantity(type=np.int32)
    area = Quantity(type=np.float64, unit='km^2')
    population_density = Quantity(type=np.float64, unit='1/km^2')

    gdp = SubSection(
        section=Timeseries,
        description='GDP per capita (constant 2005 US$)'
    )
    birth_rate = SubSection(
        section=Timeseries,
        description='per 1,000 people per year'
    )

    def normalize(self, archive, logger):
        super(Country, self).normalize(archive, logger)
        raise
        self.population_density = self.population / self.area

        self.figures.append(PlotlyFigure(
            figure=px.line(
                pd.DataFrame(dict(year=self.gdp.year, GDP=self.gdp.value)),
                x='year', y=['GDP']
            ).to_plotly_json()
        ))

save_schema_to_yaml()

You can go back to NOMAD and reprocess the upload. Look at one of the Country entries to see the plot.

<div style="height: 4rem;">&nbsp;</div>

You reached the end of this notebook. Here are some useful links:

- [nomad-lab.eu](https://nomad-lab.eu)
- [NOMAD Documentation](https://nomad-lab.eu/docs)
- [Our user forums](https://matsci.org/c/nomad/32)